This notebook shows you how you can easily load the pre-trained BERT2BERT model for empathetic response generation in Arabic and use it to generate response on your input.

In [ ]:
#Install dependencies
!pip install transformers
!pip install pyarabic
!pip install farasapy
!git clone https://github.com/aub-mind/arabert

In [ ]:
#Import transformers
import transformers

In [ ]:
#Load arabert preprocessor
from arabert.preprocess import ArabertPreprocessor
arabert_prep = ArabertPreprocessor(model_name="bert-base-arabert", keep_emojis=False)

In [ ]:
#Load Model
from transformers import EncoderDecoderModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("tareknaous/bert2bert-empathetic-response-msa")
model = EncoderDecoderModel.from_pretrained("tareknaous/bert2bert-empathetic-response-msa")

model.to("cuda")
model.eval()
print("done")

done


In [ ]:
#Function to generater response/post-process
def generate_response(text):
  text_clean = arabert_prep.preprocess(text)
  inputs = tokenizer.encode_plus(text_clean,return_tensors='pt')
  outputs = model.generate(input_ids = inputs.input_ids.to("cuda"),
                   attention_mask = inputs.attention_mask.to("cuda"),
                   num_beams=1,
                   do_sample = True,
                   min_length=10,
                   top_k = 50,
                   temperature = 1,
                   length_penalty =2)
  preds = tokenizer.batch_decode(outputs) 
  response = str(preds)
  response = response.replace("\'", '')
  response = response.replace("[[CLS]", '')
  response = response.replace("[SEP]]", '')
  response = str(arabert_prep.desegment(response))
  return response

In [ ]:
#Trial
text =  "الله يلعن هالبلد انقطعت الكهربا !"
generate_response(text)

' بالطبع ، لم يكن علي كالذهاب إلى المدرسة من قبل '